In [1]:
from utils.dataset import  Create_YOLO_Cache, YOLODataset
yaml = '/home/chaos/Documents/ChaosAIVision/Build-YOLO-v1-from-scratch/data.yaml'

/home/chaos/miniconda3/envs/chaos/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
dataset= YOLODataset('train', yaml,7,2,20,True)
a,b  = dataset.__getitem__(6)
b = b.unsqueeze(0)  # Thêm chiều batch size

print((b.shape))


torch.Size([1, 7, 7, 30])


In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Loss(nn.Module):
    def __init__(self, S, B, l_coord, l_noobj):
        super(Loss, self).__init__()
        self.S = S
        self.B = B
        self.l_coord = l_coord
        self.l_noobj = l_noobj

    def compute_iou(self, box1, box2):
        N = box1.size(0)
        M = box2.size(0)

        lt = torch.max(
            box1[:, :2].unsqueeze(1).expand(N, M, 2),
            box2[:, :2].unsqueeze(0).expand(N, M, 2),
        )

        rb = torch.min(
            box1[:, 2:].unsqueeze(1).expand(N, M, 2),
            box2[:, 2:].unsqueeze(0).expand(N, M, 2),
        )

        wh = rb - lt
        wh[wh < 0] = 0
        inter = wh[:, :, 0] * wh[:, :, 1]

        area1 = (box1[:, 2] - box1[:, 0]) * (box1[:, 3] - box1[:, 1])
        area2 = (box2[:, 2] - box2[:, 0]) * (box2[:, 3] - box2[:, 1])
        area1 = area1.unsqueeze(1).expand_as(inter)
        area2 = area2.unsqueeze(0).expand_as(inter)

        iou = inter / (area1 + area2 - inter)
        return iou

    def forward(self, prediction, target):
        device = prediction

        N = prediction.size(0)
        coo_mask = target[:, :, :, 4] > 0
        noo_mask = target[:, :, :, 4] == 0
        coo_mask = coo_mask.unsqueeze(-1).expand_as(target)
        noo_mask = noo_mask.unsqueeze(-1).expand_as(target)

        coo_pred = prediction[coo_mask].view(-1, 30)
        box_pred = coo_pred[:, :10].contiguous().view(-1, 5)
        class_pred = coo_pred[:, 10:]

        coo_target = target[coo_mask].view(-1, 30)
        box_target = coo_target[:, :10].contiguous().view(-1, 5)
        class_target = coo_target[:, 10:]

        # Đưa tất cả tensor vào đúng thiết bị
        noo_mask = noo_mask.to(device)
        noo_pred = prediction[noo_mask].view(-1, 30)
        noo_target = target[noo_mask].view(-1, 30)
        
        # Đảm bảo mặt nạ được tạo trên thiết bị đúng
        noo_pred_mask = torch.zeros(noo_pred.size(), dtype=torch.bool, device=device)
        noo_pred_mask[:, 4] = 1
        noo_pred_mask[:, 9] = 1
        noo_pred_c = noo_pred[noo_pred_mask]
        noo_target_c = noo_target[noo_pred_mask]
        nooobj_loss = F.mse_loss(noo_pred_c, noo_target_c, reduction='sum')

        coo_response_mask = torch.zeros(box_target.size(), dtype=torch.bool, device=device)
        coo_not_response_mask = torch.zeros(box_target.size(), dtype=torch.bool, device=device)
        box_target_iou = torch.zeros(box_target.size(), device=device)
        
        for i in range(0, box_target.size(0), 2):
            box1 = box_pred[i:i + 2]
            box1_xyxy = torch.zeros(box1.size(), device=device)
            box1_xyxy[:, :2] = box1[:, :2] / 14. - 0.5 * box1[:, 2:4]
            box1_xyxy[:, 2:4] = box1[:, :2] / 14. + 0.5 * box1[:, 2:4]
            box2 = box_target[i].view(-1, 5)
            box2_xyxy = torch.zeros(box2.size(), device=device)
            box2_xyxy[:, :2] = box2[:, :2] / 14. - 0.5 * box2[:, 2:4]
            box2_xyxy[:, 2:4] = box2[:, :2] / 14. + 0.5 * box2[:, 2:4]
            iou = self.compute_iou(box1_xyxy[:, :4], box2_xyxy[:, :4])
            print(iou)
            max_iou, max_index = iou.max(0)
            max_index = max_index.to(device)

            coo_response_mask[i + max_index] = 1
            coo_not_response_mask[i + 1 - max_index] = 1

            box_target_iou[i + max_index, torch.tensor([4], device=device)] = max_iou.to(device)
        
        """Loss tính toán"""
        box_pred_response = box_pred[coo_response_mask].view(-1, 5)
        box_target_response_iou = box_target_iou[coo_response_mask].view(-1, 5)
        box_target_response = box_target[coo_response_mask].view(-1, 5)
        contain_loss = F.mse_loss(box_pred_response[:, 4], box_target_response_iou[:, 4], reduction='sum')
        loc_loss = F.mse_loss(box_pred_response[:, :2], box_target_response[:, :2], reduction='sum') + F.mse_loss(
            torch.sqrt(box_pred_response[:, 2:4]), torch.sqrt(box_target_response[:, 2:4]), reduction='sum')
        
        box_pred_not_response = box_pred[coo_not_response_mask].view(-1, 5)
        box_target_not_response = box_target[coo_not_response_mask].view(-1, 5)
        box_target_not_response[:, 4] = 0

        not_contain_loss = F.mse_loss(box_pred_not_response[:, 4], box_target_not_response[:, 4], reduction='sum')

        class_loss = F.mse_loss(class_pred, class_target, reduction='sum')

        loss = self.l_coord * loc_loss + 2 * contain_loss + not_contain_loss + self.l_noobj * nooobj_loss + class_loss

        return loss / N


In [28]:
import torch

# Define parameters
S = 7  # Grid size
B = 2  # Number of bounding boxes
l_coord = 5.0  # Weight for coordinate loss
l_noobj = 0.5  # Weight for "no object" loss

# Create an instance of the Loss class
loss_fn = Loss(S=S, B=B, l_coord=l_coord, l_noobj=l_noobj)



# Compute the loss
loss = loss_fn(b, b)

print(f"Computed loss: {loss.item()}")


IndexError: tensors used as indices must be long, int, byte or bool tensors

In [29]:
import torch.nn as nn
class YOLOv1Loss(nn.Module):
    def __init__(self, S=7, B=2, C=20):
        """
        __init__ initialize YOLOv1 Loss.

        Args:
            S (int, optional): split_size. Defaults to 7.
            B (int, optional): number of boxes. Defaults to 2.
            C (int, optional): number of classes. Defaults to 20.
        """ 
        super().__init__()
        self.mse = nn.MSELoss(reduction="sum")
        self.S = S
        self.B = B
        self.C = C
        self.l_noobl = 0.5
        self.l_coord = 5

    def forward(self, predictions, target):
        predictions = predictions.reshape(-1, self.S, self.S, self.C + self.B*5)

        iou_b1 = get_iou(predictions[...,21:25], target[...,21:25])
        iou_b2 = get_iou(predictions[...,26:30], target[...,21:25])
        ious = torch.stack([iou_b1, iou_b2], 0)

        _, max_iou = torch.max(ious, dim=0)
        exists_box = target[...,20].unsqueeze(3) # select target objectness.object

        # * Box Coordinates Loss

        # Select the bounding boxes with highest IoU
        box_predictions = exists_box * (
            (
                max_iou * predictions[..., 26:30] +
                (1 - max_iou) * predictions[..., 21:25]
            )
        )

        # Select targets which has an object
        box_targets = exists_box * target[...,21:25]

        box_predictions[...,2:4] = torch.sign(box_predictions[...,2:4]) * torch.sqrt(
            torch.abs(box_predictions[..., 2:4]) + 1e-6
        )

        box_targets[..., 2:4] = torch.sqrt(box_targets[..., 2:4])

        box_loss = self.mse(
            torch.flatten(box_predictions, end_dim=-2),
            torch.flatten(box_targets, end_dim=-2)
        )

        # * Object Losss

        pred_box = (
            max_iou * predictions[..., 25:26] + 
            (1-max_iou) * predictions[..., 20:21]
        )

        object_loss = self.mse(
            torch.flatten(exists_box * pred_box),
            torch.flatten(exists_box * target[..., 20:21])
        )
        # * No Object Loss
        # For the first box
        no_boject_loss = self.mse(
            torch.flatten((1-max_iou) * predictions[...,20:21], start_dim=1),
            torch.flatten((1-max_iou) * target[...,20:21], start_dim=1)
        )
        # For the second box
        no_boject_loss += self.mse(
            torch.flatten(max_iou * predictions[...,25:26], start_dim=1),
            torch.flatten(max_iou * target[...,20:21], start_dim=1)
        )
        # * Class prediction Loss

        class_loss = self.mse(
            torch.flatten(exists_box * predictions[...,:20], end_dim=-2),
            torch.flatten(exists_box * target[...,:20], end_dim=-2)
        )

        # * Total Loss
        loss = (
            self.l_coord * box_loss 
            + object_loss
            + self.l_noobl * no_boject_loss
            + class_loss
        )

        return loss

In [30]:

import torch
def get_iou(box1, box2, box_format="midpoint"):
    """
        Calculates intersection over union
        Parameters:
            boxes1 (tensor): Predictions of Bounding Boxes (BATCH_SIZE, 4)
            box2 (tensor): Correct Labels of Boxes (BATCH_SIZE, 4)
            box_format (str): midpoint/corners, if boxes (x,y,w,h) or (x1,y1,x2,y2)
        Returns:
            tensor: Intersection over union for all examples
    """

    if box_format == "corners":
        # * box 1
        box1_x1 = box1[...,0:1]
        box1_y1 = box1[...,1:2]
        box1_x2 = box1[...,2:3]
        box1_y2 = box1[...,3:4]

        # * box 2
        box2_x1 = box2[...,0:1]
        box2_y1 = box2[...,1:2]
        box2_x2 = box2[...,2:3]
        box2_y2 = box2[...,3:4]

    elif box_format == "midpoint":
        box1_x1 = box1[..., 0:1] - box1[..., 2:3] / 2
        box1_y1 = box1[..., 1:2] - box1[..., 3:4] / 2
        box1_x2 = box1[..., 0:1] + box1[..., 2:3] / 2
        box1_y2 = box1[..., 1:2] + box1[..., 3:4] / 2
        box2_x1 = box2[..., 0:1] - box2[..., 2:3] / 2
        box2_y1 = box2[..., 1:2] - box2[..., 3:4] / 2
        box2_x2 = box2[..., 0:1] + box2[..., 2:3] / 2
        box2_y2 = box2[..., 1:2] + box2[..., 3:4] / 2
    else:
        raise NotImplementedError(f"OOPs! {box_format} not supported!")

    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)

    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)

    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1)) 

    return intersection / (box1_area + box2_area - intersection + 1e-6)

In [31]:
yolo_loss = YOLOv1Loss()
loss = yolo_loss(b, b)
print(f"Loss: {loss}")

Loss: 1.7763568394002505e-12


In [32]:
def intersection_over_union(boxes_preds, boxes_labels, box_format="midpoint"):

    """
    Calculates intersection over union

    Parameters:
        boxes_preds (tensor): Predictions of Bounding Boxes (BATCH_SIZE, 4)
        boxes_labels (tensor): Correct labels of Bounding Boxes (BATCH_SIZE, 4)
        box_format (str): midpoint/corners, if boxes (x,y,w,h) or (x1,y1,x2,y2)

    Returns:
        tensor: Intersection over union for all examples
    """

    if box_format == "midpoint":
        box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
        box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
        box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
        box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
        box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
        box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
        box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
        box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2

    if box_format == "corners":
        box1_x1 = boxes_preds[..., 0:1]
        box1_y1 = boxes_preds[..., 1:2]
        box1_x2 = boxes_preds[..., 2:3]
        box1_y2 = boxes_preds[..., 3:4]  # (N, 1)
        box2_x1 = boxes_labels[..., 0:1]
        box2_y1 = boxes_labels[..., 1:2]
        box2_x2 = boxes_labels[..., 2:3]
        box2_y2 = boxes_labels[..., 3:4]

    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)

    # .clamp(0) is for the case when they do not intersect
    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)

    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

    return intersection / (box1_area + box2_area - intersection + 1e-6)


In [33]:
class YoloLoss(nn.Module):
    """
    Calculate the loss for yolo (v1) model
    """

    def __init__(self, S=7, B=2, C=20):
        super(YoloLoss, self).__init__()
        self.mse = nn.MSELoss(reduction="sum")

        """
        S is split size of image (in paper 7),
        B is number of boxes (in paper 2),
        C is number of classes (in paper and VOC dataset is 20),
        """
        self.S = S
        self.B = B
        self.C = C

        # pay loss for no object (noobj) and the box coordinates (coord)
        self.lambda_noobj = 0.5
        self.lambda_coord = 5

    def forward(self, predictions, target):
        # predictions are shaped (BATCH_SIZE, S*S(C+B*5) when inputted
        predictions = predictions.reshape(-1, self.S, self.S, self.C + self.B * 5)

        # Calculate IoU for the two predicted bounding boxes with target bbox
        iou_b1 = intersection_over_union(predictions[..., 21:25], target[..., 21:25])
        iou_b2 = intersection_over_union(predictions[..., 26:30], target[..., 21:25])
        ious = torch.cat([iou_b1.unsqueeze(0), iou_b2.unsqueeze(0)], dim=0)

        # Take the box with highest IoU out of the two prediction
        # Note that bestbox will be indices of 0, 1 for which bbox was best
        iou_maxes, bestbox = torch.max(ious, dim=0)
        exists_box = target[..., 20].unsqueeze(3) 

        # ======================== #
        #   FOR BOX COORDINATES    #
        # ======================== #

        # Set boxes with no object in them to 0. We only take out one of the two 
        # predictions, which is the one with highest Iou calculated previously.
        box_predictions = exists_box * (
            (
                bestbox * predictions[..., 26:30]
                + (1 - bestbox) * predictions[..., 21:25]
            )
        )

        box_targets = exists_box * target[..., 21:25]

        # Take sqrt of width, height of boxes to ensure that
        box_predictions[..., 2:4] = torch.sign(box_predictions[..., 2:4]) * torch.sqrt(
            torch.abs(box_predictions[..., 2:4] + 1e-6)
        )
        box_targets[..., 2:4] = torch.sqrt(box_targets[..., 2:4])
        box_loss = self.mse(
            torch.flatten(box_predictions, end_dim=-2),
            torch.flatten(box_targets, end_dim=-2),
        )
        # ==================== #
        #   FOR OBJECT LOSS    #
        # ==================== #

        # pred_box is the confidence score for the bbox with highest IoU
        pred_box = (
            bestbox * predictions[..., 25:26] + (1 - bestbox) * predictions[..., 20:21]
        )

        object_loss = self.mse(
            torch.flatten(exists_box * pred_box),
            torch.flatten(exists_box * target[..., 20:21]),
        )

        # ======================= #
        #   FOR NO OBJECT LOSS    #
        # ======================= #



        no_object_loss = self.mse(
            torch.flatten((1 - exists_box) * predictions[..., 20:21], start_dim=1),
            torch.flatten((1 - exists_box) * target[..., 20:21], start_dim=1),
        )

        no_object_loss += self.mse(
            torch.flatten((1 - exists_box) * predictions[..., 25:26], start_dim=1),
            torch.flatten((1 - exists_box) * target[..., 20:21], start_dim=1)
        )

        # ================== #
        #   FOR CLASS LOSS   #
        # ================== #

        class_loss = self.mse(
            torch.flatten(exists_box * predictions[..., :20], end_dim=-2,),
            torch.flatten(exists_box * target[..., :20], end_dim=-2,),
        )
        

        loss = (
            self.lambda_coord * box_loss 
            + object_loss 
            + self.lambda_noobj * no_object_loss 
            + class_loss 
        )

 



        return loss 

In [34]:
yolo_loss1 = YoloLoss()


In [41]:
loss = yolo_loss1(b, b)
print(loss)

tensor(1.7764e-12)


In [39]:
class YoloLossv3(nn.Module):
    def __init__(self, S=7, B=2, C=20):
        super(YoloLossv3, self).__init__()
        self.mse = nn.MSELoss(reduction = "sum")
        self.S = S
        self.B = B
        self.C = C
        self.lambda_noobj = 0.5
        self.lambda_coord = 5

    def forward(self, predictions, target):
        predictions = predictions.reshape(-1, self.S, self.S, self.C + self.B * 5)

        iou_b1 = intersection_over_union(predictions[..., 21:25], target[..., 21:25])
        iou_b2 = intersection_over_union(predictions[..., 26:30], target[..., 21:25])
        ious = torch.cat([iou_b1.unsqueeze(0), iou_b2.unsqueeze(0)], dim=0)

        iou_maxes, bestbox = torch.max(ious, dim=0) # iou_maxes: là cái tensor có iou cao nhất, bestbox là argmax của iou cao nhất
        exists_box = target[..., 20].unsqueeze(3) # I obj_j

        # loss for box cooor - loss for box cooor - loss for box cooor 
        # box_prediction la` co box hay ko nhan cho box 1 hay 2
        box_predictions = exists_box * (
            (
                bestbox * predictions[..., 26:30]
                + (1 - bestbox) * predictions[..., 21:25]
            ) 
        )
        box_targets = exists_box * target[..., 21:25]
        
        box_predictions[..., 2:4] = torch.sign(box_predictions[..., 2:4]) * torch.sqrt(
            torch.abs(box_predictions[..., 2:4] + 1e-6)
        )
        box_targets[..., 2:4] = torch.sqrt(box_targets[..., 2:4])

        box_loss = self.mse(
            torch.flatten(box_predictions, end_dim=-2), # B,S,S,(X,Y,W,H) --> (N*S*S, 4)
            torch.flatten(box_targets, end_dim=-2),
        )

        # loss object class - loss object class - loss object class

        pred_box = (
            bestbox * predictions[..., 25:26] + (1 - bestbox) * predictions[..., 20:21]
        )
        object_loss = self.mse(
            torch.flatten(exists_box * pred_box), # (B,S,S,1) --> (B*S*S,1)
            torch.flatten(exists_box * target[..., 20:21]),
        )

        # loss no object class - loss no object class - loss no object class
        pred_no_box = (
             (1 - bestbox)  * predictions[..., 25:26] + bestbox* predictions[..., 20:21]
        )
        no_object_loss = self.mse(
            torch.flatten((1 - exists_box) * pred_no_box, start_dim=1),
            torch.flatten((1 - exists_box) * target[..., 20:21], start_dim=1),
        )


        # loss class - loss class - loss class
        class_loss = self.mse(
            torch.flatten(exists_box * predictions[..., :20], end_dim=-2,),#(N,S,S,20) --> (N*S*S, 20)
            torch.flatten(exists_box * target[..., :20], end_dim=-2,),
        )

        loss = (
            self.lambda_coord * box_loss  # first two rows in paper
            + object_loss  # third row in paper
            + self.lambda_noobj * no_object_loss  # forth row
            + class_loss  # fifth row
        )

        return loss

In [47]:
import pandas as pd
import shutil
import os

def copy_files_from_excel(image_folder, label_folder, excel_path, output_folder):
    # Đọc tệp Excel
    df = pd.read_csv(excel_path)
    
    # Tạo thư mục đầu ra nếu chưa tồn tại
    os.makedirs(os.path.join(output_folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_folder, 'labels'), exist_ok=True)
    
    for index, row in df.iterrows():
        image_file = row['image']
        label_file = row['text']
        
        # Tạo đường dẫn đầy đủ đến các tệp nguồn
        image_source_path = os.path.join(image_folder, image_file)
        label_source_path = os.path.join(label_folder, label_file)
        
        # Tạo đường dẫn đích
        image_dest_path = os.path.join(output_folder, 'images', image_file)
        label_dest_path = os.path.join(output_folder, 'labels', label_file)
        
        # Sao chép tệp ảnh nếu tồn tại
        if os.path.exists(image_source_path):
            shutil.copy(image_source_path, image_dest_path)
        else:
            print(f"Ảnh không tồn tại: {image_source_path}")
        
        # Sao chép tệp nhãn nếu tồn tại
        if os.path.exists(label_source_path):
            shutil.copy(label_source_path, label_dest_path)
        else:
            print(f"Nhãn không tồn tại: {label_source_path}")

# Ví dụ gọi hàm:
copy_files_from_excel(
    image_folder='/home/chaos/Downloads/images',
    label_folder='/home/chaos/Downloads/labels',
    excel_path='/home/chaos/Downloads/100examples.csv',
    output_folder='/home/chaos/Documents/ChaosAIVision/Build-YOLO-v1-from-scratch/dataset'
)
